# Introduction

In this notebook we will walk you through implementing a simple A* algorithm for path planning.

We have defined an abstract class for a path planning algorithm called `Graph`<sup>[1]</sup>, as well as an abstract class `Node`. Our `main` uses implementations of this interface to show the result of path planning (displaying the path and giving us its cost based on the algorithm's cost function). Your first task should be to take a look at the constructor and members of those two classes (you can ignore the helper function if you wish to).

An example implementation for a planner algorithm has been in `grass_fire.py`. You can try it out:

In [4]:
import grass_fire
grass_fire.main()

Node cost map:
1 1 1 X X X X X X 1
1 1 1 X X X X X X 1
1 1 1 X X X X X X 1
1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1
Total cost-to-go map, initial point at (1, 2)
 3.0  2.0  3.0    ?    ?    ?    ?    ?    ? 12.0
 2.0  1.0  2.0    ?    ?    ?    ?    ?    ? 11.0
 1.0    0  1.0    ?    ?    ?    ?    ?    ? 10.0
 2.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
 3.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0 10.0
Path from initial point (1, 2) to goal point (9, 0)
   X    X    X    X    X    X    X    X    X   12
   X    X    X    X    X    X    X    X    X   11
   X    0    X    X    X    X    X    X    X   10
   X    1    2    3    4    5    6    7    8    9
   X    X    X    X    X    X    X    X    X    X


However you will quickly realize that this algorithm scales very poorly to problems with bigger state space:

In [5]:
import main

main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 5, 'number_obstacles': 0, 'no_render': True})
main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 10, 'number_obstacles': 0, 'no_render': True})
main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 20, 'number_obstacles': 0, 'no_render': True})
main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 50, 'number_obstacles': 0, 'no_render': True})
main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 100, 'number_obstacles': 0, 'no_render': True})
main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 200, 'number_obstacles': 0, 'no_render': True})
main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 400, 'number_obstacles': 0, 'no_render': True})
main.run_jupy({'algo_name': 'GrassFire', 'grid_size': 500, 'number_obstacles': 0, 'no_render': True})

Path search took 0.1547169995319564ms, final cost: 8.0
Path search took 0.5831419985042885ms, final cost: 18.0
Path search took 2.577957999164937ms, final cost: 38.0
Path search took 16.07375500316266ms, final cost: 98.0
Path search took 97.99445699900389ms, final cost: 198.0
Path search took 535.8410449989606ms, final cost: 398.0
Path search took 3703.826870998455ms, final cost: 798.0
Path search took 7127.419660002488ms, final cost: 998.0


### Question 1: complexity

a. Given that our map is a square, what is the size of the input `n` based on the size of the grid `m`?

b. What seems to be the complexity of this implementation? Please note that we are lookin at `O(n)` and *not* `O(m)`.

c. Reading the source of this code (file `grass_fire.py`, function `GrassFire::compute_path`), where is the bottleneck? How could this bottleneck be optimized? What would the complexity be without this bottleneck?




# Reminder: the algorithm description

TODO: make sure those guys know wassup


# The beginning: our graph's Node class

For this exercise we give you already the definition of the `Node` we will be using. Notice the differences between `AstarNode` and `GrassFireNode`: this is why we are coupling graph creation with graph search!<sup>[1]</sup>

In [6]:
# TODO: cleanup the imports for the stoodents
from abc import ABC, abstractmethod
from math import inf
from typing import List, Optional
import itertools
from logging import debug, info, warning
from graph import Graph, Node
import heapq


class AstarNode(Node):
    def __init__(self, x: int, y: int, cost_to_go: float) -> None:
        super().__init__(x, y, cost_to_go)
        self.g = inf
        self.h = 0.0
        self.parent: Optional[AstarNode] = None

    @property
    def f(self):
        return self.h + self.cost

The `x`, `y`, and `cost_to_go` are passed to the base class. `h` is initialized to some placeholder value (could have chosen `inf` as well). The `cost` attribute of the base `Node` corresponds to the `g` function in A* parlance, hence its use in the definition of the `f` property. The `parent` attribute is used to actually construct our path: by recursively retrieving parents, one constructs a path.

## Question 2

Later in the algorithm, we will find the `AstarNode` into a priority queue (more specifically, Python's `heapq`). This will allow us to retrieve efficiently the node with the lower cost `f`). To this aim, we decide to implement a comparison operator for the node class.

a. implement the `__lt__` function below (note: this defines strict inequality `<`, not `<=`)

In [7]:
    def __lt__(self, other):
        pass # TODO

b. (answer after finishing the book) Should we (if so, why?) treat the case where `f1 = f2` but `h1 != h2 and g1 != g2`?

# Heuristic

The A* algorithm needs a heuristic `h` to direct the search. This function's goal is to give a rough estimate of the cost from a node `a` to a node `b` (usually the goal node). To allow trying out different heuristics, we created an abstract `AstarHeuristic` class:

In [8]:
class AstarHeuristic(ABC):
    @abstractmethod
    def __call__(self, xs: int, ys: int, xf: int, yf: int) -> float:
        pass

Where `xs`, `ys` and `xf`, `yf` are the coordinate of the start and end node respectively.


## Question 3

The novel idea A* brought was the use of a heuristic. From this function's properties can various properties of A* be derived. Namely, an *optimistic* heuristic guarantees the algorithm's output will be optimal. Here, *optimistic* means the value given by the function is a lower-bound to the real cost of going from one node to another, i.e.:

$$
  h~\text{is optimistic } \Leftrightarrow h(n_a, n_b) \le g^*(n_a, n_b), \forall n_a, n_b \in \mathcal{N}
$$

Where $g^*(a, b)$ is the function giving the optimal cost for going from $a$ to $b$.


a. (answer after finishing the book) why would we ever want to use a non-optimistic heuristic?

b. (answer after finishing the book) are all optimistic heuristics equivalent?

c. Implement the heuristic below

In [9]:
class Norm1AstarHeuristic(AstarHeuristic):
    pass #TODO

Looking at the `grass_fire.py` code, you may have noticed we associate to cost-to-go (i.e. the cost of going from a node A to a node B) to the destination node (i.e. we do not explictly create the edges). This association is done by passing a `cost_map` $c : (\mathbb{R} \times \mathbb{R}) \rightarrow  \mathbb{R}$ to the constructor of the graph.
This map returns the cost-to-go associated to the node (identified by its $(x, y)$ coordinates).

We do the same with our `AstarGraph`, except we create `AstarNode`s instead of `GrassFireNode`s:

In [10]:
class AstarGraph(Graph[AstarNode]):
    def __init__(self, max_x: int, max_y: int, cost_map, h: AstarHeuristic) -> None:
        super().__init__(max_x, max_y)
        self.h = h
        for z in range(max_x * max_y):
            x, y = self._unflatten(z)
            self._nodes.append(AstarNode(x, y, cost_map[(x, y)]))

Now, it is time to implement the search itself. In the block below, you will find different functions to implement. The main loop is almostly mostly yours to fill (search for the `TODO`s).

In [11]:
    def reset_nodes(self):
        pass # TODO

    def collect_path(self, node: AstarNode) -> List[AstarNode]:
        pass # TODO
    
    def get_neighbors(self, node: AstarNode) -> List[AstarNode]:
        pass # TODO

    def compute_path(self, xs: int, ys: int, xf: int, yf: int) -> List[AstarNode]:
        self.reset_nodes()
        start = self.node(xs, ys)
        start.g = 0 # cost from start to itself is obviously 0
        heap = [start]

        max_iter = 1000000
        for n in range(max_iter):
            try:
                node_to_explore = heapq.heappop(heap)
            except IndexError:
                warning(
                    f"No path exists from start ({xs}, {ys}) to goal ({xf}, {yf}) after {n+1} iterations"
                )
                return []

            x = node_to_explore.x
            y = node_to_explore.y

            if x == xf and y == yf:
                info(
                    f"Found path from start ({xs}, {ys}) to goal ({xf}, {yf}) after {n+1} iterations and {n+1+len(heap)} nodes added"
                )
                return self.collect_path(node_to_explore)

            for next_node in self.get_neighbors(node_to_explore):
                # TODO: compute costs for next_node, add it to the heap if required
                pass

        warning(f"Could not find a path after {max_iter} iterations")
        return []

a. Why do we check `x == xf and y == yf` after popping a node from the heap, rather than before inserting it (i.e. inside the `for` loop)?

# Trying out the solution

Let's try the same thing we did with the `GrassFire`, but with our new A*. This is a trivial test (since there are no obstacles), but gives us a good idea of the best-case complexity:

In [12]:
h = Norm1AstarHeuristic()
main.run_jupy({'grid_size': 5, 'number_obstacles': 0, 'no_render': True}, h)
main.run_jupy({'grid_size': 10, 'number_obstacles': 0, 'no_render': True}, h)
main.run_jupy({'grid_size': 20, 'number_obstacles': 0, 'no_render': True}, h)
main.run_jupy({'grid_size': 50, 'number_obstacles': 0, 'no_render': True}, h)
main.run_jupy({'grid_size': 100, 'number_obstacles': 0, 'no_render': True}, h)
main.run_jupy({'grid_size': 200, 'number_obstacles': 0, 'no_render': True}, h)
main.run_jupy({'grid_size': 400, 'number_obstacles': 0, 'no_render': True}, h)
main.run_jupy({'grid_size': 500, 'number_obstacles': 0, 'no_render': True}, h)

TypeError: Can't instantiate abstract class Norm1AstarHeuristic with abstract method __call__

#### Footnotes

[1] A graph and a graph search algorithm are two different things, and the latter can be implemented generically over the former. This requires however the Graph (as well as the Node and Edge) class to implement a number of functions. While not a specifically daunting task, this is not within the scope of this exercise. We have therefore decided to keep the graph creation and search tighly coupled, allowing you (almost) complete freedom over the way nodes and edges are stored. For an example implementation of a generic graph library, the Boost Graph Library (BGL) is your best bet: https://www.boost.org/doc/libs/1_81_0/libs/graph/doc/index.html